https://resanaplaza.com/2022/05/28/%E3%80%90%E5%AE%9F%E8%B7%B5%E3%80%91python%EF%BC%8Btf-idf%E3%81%A7%E6%96%87%E6%9B%B8%E3%81%AE%E7%89%B9%E5%BE%B4%E3%82%92%E8%A6%8B%E3%81%A4%E3%81%91%E3%82%8B/

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from janome.tokenizer import Tokenizer
import re
import pandas as pd
import codecs
from collections import Counter
import mojimoji

class TfIdf:
    def __init__(self):
        """
        コンストラクタ
        """

        # 分かち書きの文書を保持する変数
        self.separation_docs = {}
        self.df = None

    def create(self):
        """
        TF-IDFの実行
        """

        #TF-IDFの実行
        names,values = self.create_model(self.separation_docs.values())
        #結果をDataFrameに格納
        self.df = pd.DataFrame(values, columns = names,index = self.separation_docs.keys())

        return self.df
        
    def clean_text(self,text):
        """
        文章をクレンジング
        ・半角カタカナを全角に
        ・全角英数字を半角に
        ・大文字を小文字に
        ・不要な単語を削除

        ---------------
        Parameters:
            text : str         テキスト
        """
        text = mojimoji.han_to_zen(text, digit=False, ascii=False)
        text = mojimoji.zen_to_han(text, kana=False)
        text = text.lower()
        
        for s in ["(57)", "【要約】", "【課題】", "\r", "\n", "<br>", "（修正有）","【選択図】"]:
            text = re.sub(s, '', text)
            
        return text
    
    def word_separation(self,text):
        """
        形態素解析により一般名詞と固有名詞のリストを作成

        ---------------
        Parameters:
            text : str         テキスト
        """
        token = Tokenizer().tokenize(text)
        words = []
    
        for line in token:
            tkn = re.split('\t|,', str(line))
            # 名詞のみ対象
            if tkn[1] in ['名詞'] and tkn[2] in ['一般', '固有名詞']:
                words.append(tkn[0])
    
        return ' ' . join(words)

    def create_model(self,separating_docs):
        """
        TF-IDFの計算を行う

        ---------------
        Parameters:
            documents : [str]  分かち書きされた文書のリスト
        """      
        # モデルの生成
        vectorizer = TfidfVectorizer(smooth_idf = False, token_pattern=u'(?u)\\b\\w+\\b')
        # TF-IDF行列の計算
        values = vectorizer.fit_transform(separating_docs).toarray()
        # 特徴量ラベルの取得
        feature_names = vectorizer.get_feature_names_out()

        return feature_names,values

    def add_file(self,title,filename,encoding='utf-8'):
        '''
        ファイルの読み込み

        Parameters:
        --------
            filename : str   TF-IDFしたい文書が書かれたファイル名 
        '''
        with codecs.open(filename,'r',encoding,'ignore') as f:
            self.add_document(title,f.read())

    def add_document(self,title,document):
        '''
        テキストの読み込み

        Parameters:
        --------
            document : str   TF-IDFしたい文書
        '''
        # 形態素解析で分かち書きした文書をインスタンス変数に格納
        self.separation_docs[title] = self.word_separation(self.clean_text(document))

    def top_words(self,cnt = 10):
        '''
        TF-IDF 上位ｎの取得

        Parameters:
        --------
            cnt : int   上位からの取得件数
        '''
        res = {}
        for n,title in enumerate(self.separation_docs):
            res[title] = (self.df[n:n+1].T.sort_values(by=title, ascending=False).head(cnt)).rename(columns={title:'TFIDF'})  
        return res

    def word_count(self,cnt = 10):
        '''
        出現回数上位ｎの取得

        Parameters:
        --------
            cnt : int   上位からの取得件数
        '''
        res = {}
        for key,value in self.separation_docs.items():
            data = Counter(value.split(' ')).most_common(cnt)
            res[key] = pd.DataFrame([v for k,v in data],columns=['Count'],index=[k for k,v in data])
        return res


In [2]:
df_input = pd.read_csv('特実_国内文献.csv', encoding="utf-8")

In [3]:
tf = TfIdf()
for i in range(len(df_input)):
    title = f'{i:03}' + "_" + df_input.iloc[i]["発明の名称"] + "_" + df_input.iloc[i]["出願人/権利者"]
    document = df_input.iloc[i]["要約"]
    tf.add_document(title, document)

title_input = "入力"
document_input = "ドライバの体調異常およびその予兆を検出。カメラと運転操作による方法。"
tf.add_document(title_input, document_input)

df_tfidf_feat_vecs = tf.create()

In [4]:
# print(tf.top_words())
# print(tf.word_count())

In [5]:
import numpy as np
def cossim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [6]:
res_cossim = []
for i in range(len(df_input)):
    res_cossim.append(cossim(df_tfidf_feat_vecs.iloc[i].values, df_tfidf_feat_vecs.iloc[-1].values))
    
df_output = df_input.copy()
df_output["cossim"] = res_cossim

In [7]:
df_output.iloc[df_output.sort_values("cossim", ascending=False).index[:10]]

,文献番号,出願番号,出願日,公知日,発明の名称,出願人/権利者,FI,要約,公開番号,公告番号,登録番号,審判番号,その他,文献URL,cossim
9,特開2021-104793,特願2019-238169,2019/12/27,2021/07/26,ドライバ状態推定装置およびドライバ状態推定方法,マツダ株式会社,"B60W40/08,G08G1/16@F",(57)【要約】\r\n【課題】ドライバの運転不能につながる異常予兆を、い<BR>ち早くとら...,特開2021-104793,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.292414
8,特開2021-105958,特願2019-238171,2019/12/27,2021/07/26,ドライバ状態推定装置およびドライバ状態推定方法,マツダ株式会社,"G08G1/16@F,B60W40/08",(57)【要約】\r\n【課題】ドライバの運転不能につながる異常予兆を、い<BR>ち早く、精...,特開2021-105958,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.273519
12,特開2021-089652,特願2019-220418,2019/12/05,2021/06/10,ドライバモニタ、ドライバ状態推定方法およびプログラム,株式会社ＪＶＣケンウッド,G08G1/16@F,(57)【要約】\r\n【課題】移動体の運転者の体調に関する状態を推定する<BR>。\r\n...,特開2021-089652,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.268998
7,特開2021-138310,特願2020-039035,2020/03/06,2021/09/16,ドライバ状態推定装置およびドライバ状態推定方法,マツダ株式会社,"B60W50/08,G08G1/16@F",(57)【要約】\r\n【課題】ドライバの運転不能につながる異常予兆を、い<BR>ち早く、精...,特開2021-138310,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.244461
135,特開2005-165972,特願2003-407766,2003/12/05,2005/06/23,車線逸脱防止装置,日産自動車株式会社,"B60R1/00@A,B60R21/00,621@C,B60R21/00,624@C,B60...",(57)【要約】\r\n【課題】 ドライバに違和感を与えることなく適切なタ<BR>イミング...,特開2005-165972,NaN,特許4066942,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.162119
5,特開2021-144578,特願2020-044010,2020/03/13,2021/09/24,ドライバ状態推定装置,マツダ株式会社,"G08G1/16@F,B60W40/08,B60W50/12,G06T7/00,660@A,...",(57)【要約】\r\n 【課題】カメラとドライバとの間に偽画像を撮影させ<BR>るための...,特開2021-144578,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.153537
3,特開2021-144580,特願2020-044012,2020/03/13,2021/09/24,ドライバ状態推定装置,マツダ株式会社,"G08G1/16@F,B60W40/08,B60W50/12,G06T7/00,660@A,...",(57)【要約】\r\n 【課題】安価なシステムでもって、お面が示す人工的<BR>な顔面で...,特開2021-144580,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.121662
27,特開2019-170451,特願2018-059493,2018/03/27,2019/10/10,ドライバ状態推定装置及びドライバ状態推定方法,株式会社デンソーアイティーラボラトリ,"A61B5/18,B60R11/04,A61B5/16,200,A61B5/16,110,A...",(57)【要約】\r\n【課題】例えば車両運転中のドライバの視線を低性能の<BR>カメラで検...,特開2019-170451,NaN,特許6910982,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.098512
42,特開2017-220096,特願2016-115312,2016/06/09,2017/12/14,車両用運転支援システム,株式会社デンソー,"G08G1/16@C,B60W40/09,G08G1/16@F",(57)【要約】\r\n【課題】個々のドライバに応じた適切な運転支援の実行<BR>を可能とす...,特開2017-220096,NaN,特許6579043,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.091276
22,特開2020-077037,特願2018-208167,2018/11/05,2020/05/21,運行管理装置及び運行管理システム,株式会社デンソー,"G08G1/00@D,G07C5/00@Z,G08G1/16@F",(57)【要約】\r\n【課題】ハザードマップを生成することができる運行管<BR>理装置及び...,特開2020-077037,NaN,NaN,NaN,NaN,https://www.j-platpat.inpit.go.jp/c1800/PU/JP-...,0.085614
